In [156]:
import pandas as pd
import numpy as np
import cx_Oracle
import os
import json
import datetime
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings(action="ignore")
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import bs4
import requests
import time
import math
import random
from shapely.geometry import Point, Polygon
import time as timesys
from pyproj import Geod
from haversine import haversine
pd.set_option('display.max_columns', None)

In [112]:
import os
 
path_dir = 'voy_path'
 
file_list = os.listdir(path_dir)
file_new_list = []
for i in file_list:
    if "portmis" in i:
        file_new_list.append(i)

In [127]:
pnit_berth_history = pd.read_excel('pnit_berth_history_2.xlsx')
pnit_berth_history

,선석,선사,모선항차,선사항차,Head (Bridge) Stern,선명,ROUTE,반입마감시한,접안(예정)일시,출항(예정)일시,양하,적하,Shift,상태
0,T3(P),HMM,HHST006,0033E/0033E,41 (49) 61,HYUNDAI SATURN,AN3E,2022-07-08 22:00:00,2022-07-09 10:40:00,2022-07-11 12:00:00,1805,2999,4,DEPARTED
1,T2(P),HLC,CAUX003,2215W/2226E,23 (31) 42,CAUTIN,AN1,2022-07-09 13:00:00,2022-07-10 00:40:00,2022-07-12 05:00:00,1279,2134,896,DEPARTED
2,T1(S),MSC,MGHE003,UK221R/FI227A,21 (08) 03,MSC ALGHERO,AS2,2022-07-10 02:00:00,2022-07-10 19:25:00,2022-07-12 18:00:00,1615,1510,12,DEPARTED
3,T3(P),HLC,TAYX003,022W/022W,41 (50) 63,TAYMA EXPRESS,AG3,2022-07-11 04:00:00,2022-07-11 14:15:00,2022-07-13 11:00:00,134,2731,52,DEPARTED
4,T2(P),ONE,OAKD002,2227E/2227E,23 (36) 42,AKADIMOS,AN1,2022-07-11 19:00:00,2022-07-12 07:00:00,2022-07-13 20:00:00,1253,2032,344,DEPARTED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,T3(P),MSC,MJSS001,FA303A/FA303A,-55,MSC JOSSELINE,ASPA2E,2023-01-29 08:00:00,2023-01-29 18:00:00,2023-01-30 16:00:00,1000,2400,0,PLANNED
431,OT(P),SKR,PCSG004,/,-66,PACIFIC SINGAPORE,SETO3,2023-01-29 11:00:00,2023-01-29 21:00:00,2023-01-30 08:00:00,80,80,0,PLANNED
432,T2(P),COS,XIOZ001,059E/059E,21 (35) 41,XIN OU ZHOU,WSA3,2023-01-29 14:00:00,2023-01-30 00:00:00,2023-01-30 20:00:00,600,600,100,PLANNED
433,T1(P),COH,CFOS005,2305E/2305W,-15,COSCO FOS,AK3,2023-01-30 02:00:00,2023-01-30 12:00:00,2023-02-01 03:00:00,1700,900,0,PLANNED


In [128]:
pnit_berth_history = pnit_berth_history[pnit_berth_history["상태"] == "DEPARTED"]
pnit_berth_history

,선석,선사,모선항차,선사항차,Head (Bridge) Stern,선명,ROUTE,반입마감시한,접안(예정)일시,출항(예정)일시,양하,적하,Shift,상태
0,T3(P),HMM,HHST006,0033E/0033E,41 (49) 61,HYUNDAI SATURN,AN3E,2022-07-08 22:00:00,2022-07-09 10:40:00,2022-07-11 12:00:00,1805,2999,4,DEPARTED
1,T2(P),HLC,CAUX003,2215W/2226E,23 (31) 42,CAUTIN,AN1,2022-07-09 13:00:00,2022-07-10 00:40:00,2022-07-12 05:00:00,1279,2134,896,DEPARTED
2,T1(S),MSC,MGHE003,UK221R/FI227A,21 (08) 03,MSC ALGHERO,AS2,2022-07-10 02:00:00,2022-07-10 19:25:00,2022-07-12 18:00:00,1615,1510,12,DEPARTED
3,T3(P),HLC,TAYX003,022W/022W,41 (50) 63,TAYMA EXPRESS,AG3,2022-07-11 04:00:00,2022-07-11 14:15:00,2022-07-13 11:00:00,134,2731,52,DEPARTED
4,T2(P),ONE,OAKD002,2227E/2227E,23 (36) 42,AKADIMOS,AN1,2022-07-11 19:00:00,2022-07-12 07:00:00,2022-07-13 20:00:00,1253,2032,344,DEPARTED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,T1(P),MSC,MNIV001,HI301A/HI301A,01 (13) 18,MSC NISHA V,ORIGMA,2023-01-11 01:00:00,2023-01-11 11:00:00,2023-01-11 20:00:00,383,53,0,DEPARTED
376,T3(P),ONE,OZRE001,2301E/2301E,42 (57) 63,SEASPAN BREEZE,AN3E,2023-01-11 07:00:00,2023-01-11 17:15:00,2023-01-13 03:00:00,1407,2229,160,DEPARTED
377,T1(P),MSC,MSEN001,UX252A/UX252A,01 (14) 19,MSC ELENI,SANTA,2023-01-11 12:00:00,2023-01-11 22:25:00,2023-01-12 19:00:00,64,1567,12,DEPARTED
378,T2(P),MSC,MRGO002,FY301A/FY301A,18 (25) 40,MSC VIRGO,AFRICA,2023-01-12 01:00:00,2023-01-12 11:30:00,2023-01-13 13:00:00,1627,1247,0,DEPARTED


In [120]:
port_mis_df = pd.read_csv("port_mis_processed.csv")
port_mis_df

,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,총톤수,입항일시,출항일시,CIQ수속일자,수리일시,항해구분,MRN 번호,계선장소,계선장소.1,계선장소.2,차항지,전출항지,선박용도,외항:한국인선원수\n내항:해기사선원수,외항:외국인선원수\n내항:보통선원수,승객,예선,도선,부선호출부호1,부선호출부호2
0,부산,VRAD2,OSG ALPHA,2015,8,최종,외항,출항,"7,167",2015-09-24 16:00,2022-08-10 00:00,2015-09-24 19:00,2015-10-06 11:27,기타,15ANJSM0229,WAM,08,M박지 M8 묘지,DALIAN,MAIZURU,풀컨테이너선,NaN,17.0,NaN,N,N,NaN,NaN
1,부산,140025,한림,2015,23,최종,내항,출항,432,2015-09-25 11:40,2022-07-31 00:00,NaN,2015-10-08 10:25,기타,NaN,WAN,01,남외항 N-1박지,온산,제주,석유제품 운반선,5.0,2.0,NaN,N,N,NaN,NaN
2,부산,030968,삼원골드,2017,93,변경,내항,출항,790,2017-11-01 04:50,2023-01-11 23:00,NaN,NaN,기타,NaN,MW5,01,5물량장 1선석,기타항,기타항,석유제품 운반선,4.0,4.0,0.0,N,N,NaN,NaN
3,부산,D9JC,성신티3호,2019,7,최종,내항,출항,106.21,2019-03-21 12:50,2022-11-01 13:35,NaN,2022-11-02 09:45,기타,NaN,*LP,01,대평동호안,기타항,기타항,기타 예선,4.0,0.0,0.0,N,N,NaN,NaN
4,부산,UCXC,CAESAR,2019,2,변경,외항,출항,685,2019-12-29 15:17,2022-12-31 18:00,2022-12-31 18:00,NaN,기타,21TNICD005E,MYQ,01,기타 북항사설조선소,OCEAN DISTRICT,속초,냉동.냉장선,NaN,2.0,NaN,Y,Y,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42633,부산,220031,말리,2023,1,최초,내항,입항,"2,692",2023-01-13 21:00,2023-01-14 05:00,NaN,NaN,기타,NaN,MBE,Q1,감만부두 동측안벽 1 선석,울산,대산,석유제품 운반선,8.0,8.0,0.0,Y,N,NaN,NaN
42634,부산,9HA4999,CMA CGM J ADAMS,2023,1,최종,외항,입항,"141,950",2023-01-13 21:00,2023-01-14 19:00,2023-01-13 21:00,NaN,기타,23CMAL0004I,MS5,04,신항 5부두 4선석,SHEKOU PT,YOKOHAMA,풀컨테이너선,NaN,28.0,NaN,Y,Y,NaN,NaN
42635,부산,A8SI7,ZIM SAN DIEGO,2023,1,최종,외항,입항,"91,158",2023-01-13 21:00,2023-01-14 19:00,2023-01-13 21:00,NaN,기타,23ZIMU0020I,MSN,05,신항 2부두 2선석,"HOUSTON, TX",YANTIAN PT,풀컨테이너선,NaN,20.0,NaN,Y,Y,NaN,NaN
42636,부산,3FNR5,DONG YU,2023,1,변경,외항,입항,"2,962",2023-01-13 23:00,2023-01-14 12:00,2023-01-13 23:30,NaN,기타,23SHSBM011I,WAN,03,남외항 N-3박지,SAKAISENBOKU,RIZHAO PT,일반화물선,NaN,13.0,NaN,N,N,NaN,NaN


In [132]:
pnit_berth_history[pnit_berth_history["선명"] == 'SEASPAN BRIGHTNESS']

,선석,선사,모선항차,선사항차,Head (Bridge) Stern,선명,ROUTE,반입마감시한,접안(예정)일시,출항(예정)일시,양하,적하,Shift,상태
20,T1(P),ONE,ORGS004,2228E/2228E,01 (16) 22,SEASPAN BRIGHTNESS,AN1,2022-07-20 01:00:00,2022-07-20 11:00:00,2022-07-22 05:00:00,1624,2396,6,DEPARTED
165,T3(P),ONE,ORGS005,2239E/2239E,42 (57) 63,SEASPAN BRIGHTNESS,AN1,2022-10-04 16:00:00,2022-10-05 02:25:00,2022-10-06 12:00:00,1263,1470,16,DEPARTED
335,T3(P),ONE,ORGS006,2251E/2251E,42 (57) 63,SEASPAN BRIGHTNESS,AN1,2022-12-25 14:00:00,2022-12-26 00:20:00,2022-12-27 02:00:00,515,2335,2,DEPARTED


In [125]:
for name in set(pnit_berth_history["선명"]):
    display(port_mis_df[port_mis_df["선명"] == name])

,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,총톤수,입항일시,출항일시,CIQ수속일자,수리일시,항해구분,MRN 번호,계선장소,계선장소.1,계선장소.2,차항지,전출항지,선박용도,외항:한국인선원수\n내항:해기사선원수,외항:외국인선원수\n내항:보통선원수,승객,예선,도선,부선호출부호1,부선호출부호2
9134,부산,5LCL2,MSC NASSAU,2022,8,최종,외항,입항,"40,993",2022-08-24 18:20,2022-08-25 19:00,2022-08-24 18:00,2022-08-25 10:20,기타,22MSCU2124I,MSN,01,신항 1부두 1선석,CRISTOBAL,NINGBO,풀컨테이너선,NaN,24.0,NaN,Y,Y,NaN,NaN
9135,부산,5LCL2,MSC NASSAU,2022,8,최종,외항,출항,"40,993",2022-08-24 18:20,2022-08-25 19:00,2022-08-25 19:00,2022-08-26 10:04,기타,22MSCU0974E,MSN,01,신항 1부두 1선석,CRISTOBAL,NINGBO,풀컨테이너선,NaN,27.0,NaN,Y,Y,NaN,NaN


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,총톤수,입항일시,출항일시,CIQ수속일자,수리일시,항해구분,MRN 번호,계선장소,계선장소.1,계선장소.2,차항지,전출항지,선박용도,외항:한국인선원수\n내항:해기사선원수,외항:외국인선원수\n내항:보통선원수,승객,예선,도선,부선호출부호1,부선호출부호2
755,부산,VRNL9,SEASPAN BRIGHTNESS,2022,3,최종,외항,입항,"113,042",2022-07-20 10:50,2022-07-22 05:22,2022-07-20 11:00,2022-07-22 13:10,기타,22ONEY0865I,MSN,01,신항 1부두 1선석,MANZANILLO,NINGBO,풀컨테이너선,NaN,21.0,NaN,Y,Y,NaN,NaN
756,부산,VRNL9,SEASPAN BRIGHTNESS,2022,3,최종,외항,출항,"113,042",2022-07-20 10:50,2022-07-22 05:22,2022-07-22 05:00,2022-07-25 09:09,기타,22ONEY0866E,MSN,01,신항 1부두 1선석,MANZANILLO,NINGBO,풀컨테이너선,NaN,20.0,NaN,Y,Y,NaN,NaN
18312,부산,VRNL9,SEASPAN BRIGHTNESS,2022,4,최종,외항,입항,"113,042",2022-10-05 02:13,2022-10-06 12:03,2022-10-05 02:00,2022-10-06 09:04,기타,22ONEY1179I,MSN,03,신항 1부두 3선석,MANZANILLO,NINGBO,풀컨테이너선,NaN,21.0,NaN,Y,Y,NaN,NaN
18313,부산,VRNL9,SEASPAN BRIGHTNESS,2022,4,최종,외항,출항,"113,042",2022-10-05 02:13,2022-10-06 12:03,2022-10-06 11:00,2022-10-07 08:34,기타,22ONEY1180E,MSN,03,신항 1부두 3선석,MANZANILLO,NINGBO,풀컨테이너선,NaN,21.0,NaN,Y,Y,NaN,NaN
38205,부산,VRNL9,SEASPAN BRIGHTNESS,2022,5,최종,외항,입항,"113,042",2022-12-26 00:00,2022-12-27 02:03,2022-12-26 00:01,2022-12-27 09:41,기타,22ONEY1519I,MSN,03,신항 1부두 3선석,MANZANILLO,NINGBO,풀컨테이너선,NaN,21.0,NaN,Y,Y,NaN,NaN
38206,부산,VRNL9,SEASPAN BRIGHTNESS,2022,5,최종,외항,출항,"113,042",2022-12-26 00:00,2022-12-27 02:03,2022-12-27 02:00,2022-12-27 14:18,기타,22ONEY1520E,MSN,03,신항 1부두 3선석,MANZANILLO,NINGBO,풀컨테이너선,NaN,24.0,NaN,Y,Y,NaN,NaN


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,총톤수,입항일시,출항일시,CIQ수속일자,수리일시,항해구분,MRN 번호,계선장소,계선장소.1,계선장소.2,차항지,전출항지,선박용도,외항:한국인선원수\n내항:해기사선원수,외항:외국인선원수\n내항:보통선원수,승객,예선,도선,부선호출부호1,부선호출부호2
3037,부산,D5WP9,MSC ANDREA F,2022,2,최종,외항,입항,"9,957",2022-07-29 19:55,2022-07-30 14:54,2022-07-29 20:00,2022-08-01 08:21,기타,22MSCU4242I,MSN,01,신항 1부두 1선석,PETROPAVLOVSK-KAMCHATSKIY,PETROPAVLOVSK-KAMCHATSKIY,풀컨테이너선,NaN,15.0,NaN,Y,Y,NaN,NaN
3038,부산,D5WP9,MSC ANDREA F,2022,2,최종,외항,출항,"9,957",2022-07-29 19:55,2022-07-30 14:54,2022-07-30 15:00,2022-08-01 08:29,기타,22MSCU0698E,MSN,01,신항 1부두 1선석,PETROPAVLOVSK-KAMCHATSKIY,PETROPAVLOVSK-KAMCHATSKIY,풀컨테이너선,NaN,15.0,NaN,Y,Y,NaN,NaN
7800,부산,D5WP9,MSC ANDREA F,2022,3,최종,외항,입항,"9,957",2022-08-19 02:35,2022-08-19 11:50,2022-08-19 02:00,2022-08-22 08:20,기타,22MSCU4261I,MSN,02,신항 1부두 2선석,PETROPAVLOVSK-KAMCHATSKIY,PETROPAVLOVSK-KAMCHATSKIY,풀컨테이너선,NaN,15.0,NaN,Y,Y,NaN,NaN
7801,부산,D5WP9,MSC ANDREA F,2022,3,최종,외항,출항,"9,957",2022-08-19 02:35,2022-08-19 11:50,2022-08-19 12:00,2022-08-22 09:26,기타,22MSCU0928E,MSN,02,신항 1부두 2선석,PETROPAVLOVSK-KAMCHATSKIY,PETROPAVLOVSK-KAMCHATSKIY,풀컨테이너선,NaN,17.0,NaN,Y,Y,NaN,NaN
13590,부산,D5WP9,MSC ANDREA F,2022,4,최종,외항,출항,"9,957",2022-09-15 04:07,2022-09-15 18:01,2022-09-15 17:00,2022-09-16 10:54,기타,22MSCU1026E,MSN,01,신항 1부두 1선석,PETROPAVLOVSK-KAMCHATSKIY,PETROPAVLOVSK-KAMCHATSKIY,풀컨테이너선,NaN,17.0,NaN,Y,Y,NaN,NaN
13591,부산,D5WP9,MSC ANDREA F,2022,4,최종,외항,입항,"9,957",2022-09-15 04:07,2022-09-15 18:01,2022-09-15 04:00,2022-09-15 09:08,기타,22MSCU4283I,MSN,01,신항 1부두 1선석,PETROPAVLOVSK-KAMCHATSKIY,PETROPAVLOVSK-KAMCHATSKIY,풀컨테이너선,NaN,17.0,NaN,Y,Y,NaN,NaN
18298,부산,D5WP9,MSC ANDREA F,2022,5,최종,외항,입항,"9,957",2022-10-05 00:01,2022-10-05 08:15,2022-10-05 00:00,2022-10-05 08:57,기타,22MSCU4289I,MSN,01,신항 1부두 1선석,PETROPAVLOVSK-KAMCHATSKIY,PETROPAVLOVSK-KAMCHATSKIY,풀컨테이너선,NaN,17.0,NaN,Y,Y,NaN,NaN
18299,부산,D5WP9,MSC ANDREA F,2022,5,최종,외항,출항,"9,957",2022-10-05 00:01,2022-10-05 08:15,2022-10-05 08:00,2022-10-05 10:24,기타,22MSCU1102E,MSN,01,신항 1부두 1선석,PETROPAVLOVSK-KAMCHATSKIY,PETROPAVLOVSK-KAMCHATSKIY,풀컨테이너선,NaN,17.0,NaN,Y,Y,NaN,NaN
23338,부산,D5WP9,MSC ANDREA F,2022,6,최종,외항,입항,"9,957",2022-10-26 00:00,2022-10-26 12:00,2022-10-26 00:00,2022-10-26 10:43,기타,22MSCU4315I,MSN,03,신항 1부두 3선석,PETROPAVLOVSK-KAMCHATSKIY,VLADIVOSTOK,풀컨테이너선,NaN,17.0,NaN,Y,Y,NaN,NaN
23339,부산,D5WP9,MSC ANDREA F,2022,6,최종,외항,출항,"9,957",2022-10-26 00:00,2022-10-26 12:00,2022-10-26 12:00,2022-10-26 13:28,기타,22MSCU1147E,MSN,03,신항 1부두 3선석,PETROPAVLOVSK-KAMCHATSKIY,VLADIVOSTOK,풀컨테이너선,NaN,17.0,NaN,Y,Y,NaN,NaN


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,총톤수,입항일시,출항일시,CIQ수속일자,수리일시,항해구분,MRN 번호,계선장소,계선장소.1,계선장소.2,차항지,전출항지,선박용도,외항:한국인선원수\n내항:해기사선원수,외항:외국인선원수\n내항:보통선원수,승객,예선,도선,부선호출부호1,부선호출부호2
14550,부산,VRJS2,OOCL NEW YORK,2022,1,최종,외항,입항,"66,289",2022-09-21 06:04,2022-09-21 16:38,2022-09-21 06:04,2022-09-22 10:10,기타,22OOLU0161I,MSN,03,신항 1부두 3선석,KAOHSIUNG,TIANJIN XINGANG PT,풀컨테이너선,NaN,28.0,NaN,Y,Y,NaN,NaN
14551,부산,VRJS2,OOCL NEW YORK,2022,1,최종,외항,출항,"66,289",2022-09-21 06:04,2022-09-21 16:38,2022-09-21 16:38,2022-09-22 10:32,기타,22OOLU0162E,MSN,03,신항 1부두 3선석,SHANGHAI,TIANJIN XINGANG PT,풀컨테이너선,NaN,28.0,NaN,Y,Y,NaN,NaN
27568,부산,VRJS2,OOCL NEW YORK,2022,2,최종,외항,출항,"66,289",2022-11-11 14:10,2022-11-12 03:08,2022-11-12 03:08,2022-11-14 13:17,기타,22OOLU0204E,MSN,01,신항 1부두 1선석,SHANGHAI,TIANJIN XINGANG PT,풀컨테이너선,NaN,27.0,NaN,Y,Y,NaN,NaN
27569,부산,VRJS2,OOCL NEW YORK,2022,2,최종,외항,입항,"66,289",2022-11-11 14:10,2022-11-12 03:08,2022-11-11 14:10,2022-11-14 11:02,기타,22OOLU0203I,MSN,01,신항 1부두 1선석,SHANGHAI,TIANJIN XINGANG PT,풀컨테이너선,NaN,28.0,NaN,Y,Y,NaN,NaN
39759,부산,VRJS2,OOCL NEW YORK,2022,3,최종,외항,출항,"66,289",2022-12-31 19:40,2023-01-01 10:52,2023-01-01 10:52,2023-01-02 10:21,기타,23OOLU9906E,MSN,01,신항 1부두 1선석,SHANGHAI,TIANJIN XINGANG PT,풀컨테이너선,NaN,26.0,NaN,Y,Y,NaN,NaN
39760,부산,VRJS2,OOCL NEW YORK,2022,3,최종,외항,입항,"66,289",2022-12-31 19:40,2023-01-01 10:52,2022-12-31 19:40,2023-01-02 10:02,기타,22OOLU0249I,MSN,01,신항 1부두 1선석,SHANGHAI,TIANJIN XINGANG PT,풀컨테이너선,NaN,26.0,NaN,Y,Y,NaN,NaN


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,총톤수,입항일시,출항일시,CIQ수속일자,수리일시,항해구분,MRN 번호,계선장소,계선장소.1,계선장소.2,차항지,전출항지,선박용도,외항:한국인선원수\n내항:해기사선원수,외항:외국인선원수\n내항:보통선원수,승객,예선,도선,부선호출부호1,부선호출부호2
6796,부산,5LAK7,MSC ABY,2022,1,최종,외항,입항,"92,198",2022-08-14 14:10,2022-08-15 23:00,2022-08-14 14:00,2022-08-16 08:45,기타,22MSCU4222I,MSN,03,신항 1부두 3선석,MANZANILLO,SHANGHAI,풀컨테이너선,NaN,27.0,NaN,Y,Y,NaN,NaN
6797,부산,5LAK7,MSC ABY,2022,1,최종,외항,출항,"92,198",2022-08-14 14:10,2022-08-15 23:00,2022-08-15 22:00,2022-08-16 08:54,기타,22MSCU0694E,MSN,03,신항 1부두 3선석,MANZANILLO,SHANGHAI,풀컨테이너선,NaN,28.0,NaN,Y,Y,NaN,NaN


...

,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,총톤수,입항일시,출항일시,CIQ수속일자,수리일시,항해구분,MRN 번호,계선장소,계선장소.1,계선장소.2,차항지,전출항지,선박용도,외항:한국인선원수\n내항:해기사선원수,외항:외국인선원수\n내항:보통선원수,승객,예선,도선,부선호출부호1,부선호출부호2
8490,부산,3ELQ8,COSCO ASIA,2022,1,최종,외항,입항,"114,394",2022-08-22 05:10,2022-08-22 17:00,2022-08-22 05:10,2022-08-22 10:34,기타,22COSU0144I,MSN,03,신항 1부두 3선석,LAZARO CARDENAS,QINGDAO,풀컨테이너선,NaN,27.0,NaN,Y,Y,NaN,NaN
8491,부산,3ELQ8,COSCO ASIA,2022,1,최종,외항,출항,"114,394",2022-08-22 05:10,2022-08-22 17:00,2022-08-22 17:00,2022-08-23 10:18,기타,22COSU0144E,MSN,03,신항 1부두 3선석,LAZARO CARDENAS,QINGDAO,풀컨테이너선,NaN,27.0,NaN,Y,Y,NaN,NaN
26848,부산,3ELQ8,COSCO ASIA,2022,2,최종,외항,출항,"114,394",2022-11-08 17:20,2022-11-09 12:59,2022-11-09 12:59,2022-11-10 14:02,기타,22COSU0183E,MSN,02,신항 1부두 2선석,ENSENADA,QINGDAO,풀컨테이너선,NaN,27.0,NaN,Y,Y,NaN,NaN
26849,부산,3ELQ8,COSCO ASIA,2022,2,최종,외항,입항,"114,394",2022-11-08 17:20,2022-11-09 12:59,2022-11-08 17:20,2022-11-09 13:12,기타,22COSU0183I,MSN,02,신항 1부두 2선석,LAZARO CARDENAS,QINGDAO,풀컨테이너선,NaN,27.0,NaN,Y,Y,NaN,NaN


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,총톤수,입항일시,출항일시,CIQ수속일자,수리일시,항해구분,MRN 번호,계선장소,계선장소.1,계선장소.2,차항지,전출항지,선박용도,외항:한국인선원수\n내항:해기사선원수,외항:외국인선원수\n내항:보통선원수,승객,예선,도선,부선호출부호1,부선호출부호2
2023,부산,VRAB8,CSCL ASIA,2022,3,최종,외항,출항,"90,645",2022-07-26 00:20,2022-07-26 16:53,2022-07-26 16:53,2022-07-29 09:23,기타,22COSU0131E,MSN,01,신항 1부두 1선석,ENSENADA,QINGDAO,풀컨테이너선,NaN,25.0,NaN,Y,Y,NaN,NaN
2024,부산,VRAB8,CSCL ASIA,2022,3,최종,외항,입항,"90,645",2022-07-26 00:20,2022-07-26 16:53,2022-07-26 00:20,2022-07-26 10:24,기타,22COSU0127I,MSN,01,신항 1부두 1선석,ENSENADA,QINGDAO,풀컨테이너선,NaN,25.0,NaN,Y,Y,NaN,NaN
19908,부산,VRAB8,CSCL ASIA,2022,4,최종,외항,출항,"90,645",2022-10-12 06:05,2022-10-13 04:54,2022-10-13 04:54,2022-10-13 09:53,기타,22COSU0166E,MSN,01,신항 1부두 1선석,ENSENADA,QINGDAO,풀컨테이너선,NaN,28.0,NaN,Y,Y,NaN,NaN
19909,부산,VRAB8,CSCL ASIA,2022,4,최종,외항,입항,"90,645",2022-10-12 06:05,2022-10-13 04:54,2022-10-12 06:05,2022-10-12 10:21,기타,22COSU0166I,MSN,01,신항 1부두 1선석,ENSENADA,QINGDAO,풀컨테이너선,NaN,28.0,NaN,Y,Y,NaN,NaN
39881,부산,VRAB8,CSCL ASIA,2023,1,최종,외항,입항,"90,645",2023-01-01 07:00,2023-01-01 19:50,2023-01-01 07:00,2023-01-02 09:59,기타,23COSU0006I,MSN,02,신항 1부두 2선석,ENSENADA,QINGDAO,풀컨테이너선,NaN,28.0,NaN,Y,Y,NaN,NaN
39882,부산,VRAB8,CSCL ASIA,2023,1,최종,외항,출항,"90,645",2023-01-01 07:00,2023-01-01 19:50,2023-01-01 19:50,2023-01-02 13:03,기타,23COSU0006E,MSN,02,신항 1부두 2선석,ENSENADA,QINGDAO,풀컨테이너선,NaN,28.0,NaN,Y,Y,NaN,NaN


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,총톤수,입항일시,출항일시,CIQ수속일자,수리일시,항해구분,MRN 번호,계선장소,계선장소.1,계선장소.2,차항지,전출항지,선박용도,외항:한국인선원수\n내항:해기사선원수,외항:외국인선원수\n내항:보통선원수,승객,예선,도선,부선호출부호1,부선호출부호2
1547,부산,D5FL9,MSC SOFIA PAZ,2022,1,최종,외항,입항,"94,930",2022-07-23 20:03,2022-07-25 01:05,2022-07-23 20:00,2022-07-25 09:08,기타,22MSCU4232I,MSN,01,신항 1부두 1선석,"CHARLESTON, SC",SHANGHAI,풀컨테이너선,NaN,21.0,NaN,Y,Y,NaN,NaN
1548,부산,D5FL9,MSC SOFIA PAZ,2022,1,최종,외항,출항,"94,930",2022-07-23 20:03,2022-07-25 01:05,2022-07-25 01:00,2022-07-25 09:15,기타,22MSCU0695E,MSN,01,신항 1부두 1선석,"CHARLESTON, SC",SHANGHAI,풀컨테이너선,NaN,24.0,NaN,Y,Y,NaN,NaN
24513,부산,D5FL9,MSC SOFIA PAZ,2022,2,최종,외항,입항,"94,930",2022-10-30 09:03,2022-10-31 03:20,2022-10-30 09:00,2022-10-31 09:18,기타,22MSCU1213I,MSN,01,신항 1부두 1선석,광양,TIANJIN XINGANG PT,풀컨테이너선,NaN,26.0,NaN,Y,Y,NaN,NaN
24514,부산,D5FL9,MSC SOFIA PAZ,2022,2,최종,외항,출항,"94,930",2022-10-30 09:03,2022-10-31 03:20,2022-10-31 03:00,2022-10-31 09:42,기타,22MSCU2076E,MSN,01,신항 1부두 1선석,광양,TIANJIN XINGANG PT,풀컨테이너선,NaN,23.0,NaN,Y,Y,NaN,NaN


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,총톤수,입항일시,출항일시,CIQ수속일자,수리일시,항해구분,MRN 번호,계선장소,계선장소.1,계선장소.2,차항지,전출항지,선박용도,외항:한국인선원수\n내항:해기사선원수,외항:외국인선원수\n내항:보통선원수,승객,예선,도선,부선호출부호1,부선호출부호2
3809,부산,9V7282,AKA BHUM,2022,5,최종,외항,출항,"66,462",2022-08-02 12:30,2022-08-03 08:05,2022-08-03 07:30,2022-08-03 09:18,기타,22CJBV0040E,MSN,02,신항 1부두 2선석,SHEKOU PT,TIANJIN XINGANG PT,풀컨테이너선,NaN,22.0,NaN,Y,Y,NaN,NaN
3810,부산,9V7282,AKA BHUM,2022,5,최종,외항,입항,"66,462",2022-08-02 12:30,2022-08-03 08:05,2022-08-02 13:00,2022-08-02 14:03,기타,22CJBV0039I,MSN,02,신항 1부두 2선석,SHEKOU PT,TIANJIN XINGANG PT,풀컨테이너선,NaN,22.0,NaN,Y,Y,NaN,NaN
13240,부산,9V7282,AKA BHUM,2022,6,최종,외항,출항,"66,462",2022-09-13 11:00,2022-09-14 02:58,2022-09-14 02:30,2022-09-14 09:10,기타,22CJBV0046E,MSN,01,신항 1부두 1선석,SHANGHAI,TIANJIN XINGANG PT,풀컨테이너선,NaN,21.0,NaN,Y,Y,NaN,NaN
13241,부산,9V7282,AKA BHUM,2022,6,최종,외항,입항,"66,462",2022-09-13 11:00,2022-09-14 02:58,2022-09-13 11:00,2022-09-13 15:41,기타,22CJBV0045I,MSN,01,신항 1부두 1선석,SHEKOU PT,TIANJIN XINGANG PT,풀컨테이너선,NaN,21.0,NaN,Y,Y,NaN,NaN
26173,부산,9V7282,AKA BHUM,2022,7,최종,외항,입항,"66,462",2022-11-05 16:30,2022-11-06 05:59,2022-11-05 17:00,2022-11-07 09:15,기타,22CJBV0053I,MSN,02,신항 1부두 2선석,SHANGHAI,TIANJIN XINGANG PT,풀컨테이너선,NaN,22.0,NaN,Y,Y,NaN,NaN
26174,부산,9V7282,AKA BHUM,2022,7,최종,외항,출항,"66,462",2022-11-05 16:30,2022-11-06 05:59,2022-11-06 05:30,2022-11-07 09:27,기타,22CJBV0054E,MSN,02,신항 1부두 2선석,SHANGHAI,TIANJIN XINGANG PT,풀컨테이너선,NaN,22.0,NaN,Y,Y,NaN,NaN
38519,부산,9V7282,AKA BHUM,2022,8,최종,외항,출항,"66,462",2022-12-27 06:55,2022-12-28 00:15,2022-12-28 00:00,2022-12-28 09:23,기타,22CJBV0062E,MSN,01,신항 1부두 1선석,SHANGHAI,TIANJIN XINGANG PT,풀컨테이너선,NaN,23.0,NaN,Y,Y,NaN,NaN
38520,부산,9V7282,AKA BHUM,2022,8,최종,외항,입항,"66,462",2022-12-27 06:55,2022-12-28 00:15,2022-12-27 07:00,2022-12-27 09:41,기타,22CJBV0061I,MSN,01,신항 1부두 1선석,SHANGHAI,TIANJIN XINGANG PT,풀컨테이너선,NaN,23.0,NaN,Y,Y,NaN,NaN


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,총톤수,입항일시,출항일시,CIQ수속일자,수리일시,항해구분,MRN 번호,계선장소,계선장소.1,계선장소.2,차항지,전출항지,선박용도,외항:한국인선원수\n내항:해기사선원수,외항:외국인선원수\n내항:보통선원수,승객,예선,도선,부선호출부호1,부선호출부호2
2239,부산,A8HZ5,NAVIOS UTMOST,2022,2,최종,외항,입항,"90,389",2022-07-26 19:02,2022-07-27 20:00,2022-07-26 19:00,2022-07-27 09:13,기타,22MSCU4244I,MSN,01,신항 1부두 1선석,SEATTLE,DA CHAN BAY,풀컨테이너선,NaN,22.0,NaN,Y,Y,NaN,NaN
2240,부산,A8HZ5,NAVIOS UTMOST,2022,2,최종,외항,출항,"90,389",2022-07-26 19:02,2022-07-27 20:00,2022-07-27 20:00,2022-07-28 08:43,기타,22MSCU0687E,MSN,01,신항 1부두 1선석,LONGBEACH,DA CHAN BAY,풀컨테이너선,NaN,22.0,NaN,Y,Y,NaN,NaN


In [133]:
file_list

['AKADIMOS_9HA5144_2022_3.csv',
 'AKADIMOS_portmis.csv',
 'AKA_BHUM_9V7282_2022_5.csv',
 'AKA_BHUM_9V7282_2022_6.csv',
 'AKA_BHUM_9V7282_2022_7.csv',
 'AKA_BHUM_9V7282_2022_8.csv',
 'AKA_BHUM_9V7282_6.csv',
 'AKA_BHUM_9V7282_7.csv',
 'AKA_BHUM_portmis.csv',
 'AMALTHEA_CQDE_2022_1.csv',
 'AMALTHEA_portmis.csv',
 'CAPE_ARTEMISIO_9HA4529_2022_4.csv',
 'CAPE_ARTEMISIO_portmis.csv',
 'CAUQUENES_D5HF2_2022_3.csv',
 'CAUQUENES_D5HF2_2022_4.csv',
 'CAUQUENES_portmis.csv',
 'CAUTIN_D5HE8_2022_4.csv',
 'CAUTIN_D5HE8_2022_5.csv',
 'CAUTIN_portmis.csv',
 'CISNES_D5HF4_2022_3.csv',
 'CISNES_D5HF4_2022_4.csv',
 'CISNES_portmis.csv',
 'COCHRANE_D5HE9_2022_3.csv',
 'COCHRANE_D5HE9_2022_4.csv',
 'COCHRANE_D5HE9_4.csv',
 'COCHRANE_portmis.csv',
 'COPIAPO_D5HE7_2022_4.csv',
 'COPIAPO_D5HE7_2022_5.csv',
 'COPIAPO_D5HE7_5.csv',
 'COPIAPO_portmis.csv',
 'COSCO_ASIA_3ELQ8_2.csv',
 'COSCO_ASIA_3ELQ8_2022_1.csv',
 'COSCO_ASIA_3ELQ8_2022_2.csv',
 'COSCO_ASIA_portmis.csv',
 'COSCO_FOS_portmis.csv',
 'COSCO_FOS_V

# 각 선박들 별로 부산 polygon 확인

In [144]:
for i in file_list[:3]:
    if 'upper_china' in i or 'portmis' in i or 'test' in i:
        continue 
    df = pd.read_csv(f"voy_path/{i}")
    if len(df) == 0:
        continue
    df = df.loc[df[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
    df['time_position_data_received'] = pd.to_datetime(df['time_position_data_received'])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.sort_values(by="timestamp", ignore_index=True,inplace=True)
    print(i)
    display(df)

AKADIMOS_9HA5144_2022_3.csv


,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
0,AKADIMOS,215499000,9706308.0,9288,HKHKG,9140400,141.5711,13.4902,16.9,290.0,2022-09-11 00:10:21.029964,2022-09-10 13:39:31,2022-09-10 09:40:27,under way using engine,2709.886309,8042.979719
1,AKADIMOS,215499000,9706308.0,9288,HKHKG,9140400,133.1867,16.3133,18.0,NaN,2022-09-12 10:23:06.653045,2022-09-11 18:59:19,2022-09-10 09:40:27,under way using engine,2119.497891,6708.438624
2,AKADIMOS,215499000,9706308.0,9288,HKHKG,9150900,119.6501,19.7207,10.0,303.0,2022-09-14 21:35:01.165913,2022-09-14 11:20:12,2022-09-14 11:21:15,under way using engine,1919.934315,4650.832680
3,AKADIMOS,215499000,9706308.0,9288,HKHKG,9150900,118.7239,20.2812,11.7,303.0,2022-09-15 02:06:49.165779,2022-09-14 16:56:52,2022-09-14 04:57:16,under way using engine,1909.546525,4619.020125
4,AKADIMOS,215499000,9706308.0,9288,HKHKG,9150900,118.6643,20.3166,12.1,303.0,2022-09-15 02:36:52.010736,2022-09-14 17:16:54,2022-09-14 05:21:16,under way using engine,1909.143750,4647.113511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3320,AKADIMOS,215499000,9706308.0,9288,KRBNP,12181045,128.8294,35.0773,0.0,90.0,2022-12-18 23:22:11.864978,2022-12-18 14:18:28,2022-12-18 02:21:33.000000,moored,12.009935,NaN
3321,AKADIMOS,215499000,9706308.0,9288,KRBNP,12181045,128.8294,35.0774,0.0,90.0,2022-12-18 23:32:12.413100,2022-12-18 14:27:29,2022-12-18 02:27:35.000000,moored,12.021023,NaN
3322,AKADIMOS,215499000,9706308.0,9288,KRBNP,12181045,128.8294,35.0773,0.0,90.0,2022-12-18 23:37:14.034124,2022-12-18 14:36:33,2022-12-18 02:27:35.000000,moored,12.009935,NaN
3323,AKADIMOS,215499000,9706308.0,9288,KRBNP,12181045,128.8295,35.0773,0.0,90.0,2022-12-18 23:52:13.071206,2022-12-18 14:48:36,2022-12-18 02:51:36.000000,moored,12.010626,NaN


In [148]:
port_mis_list = []
for i in file_list:
    if 'portmis' in i:
        port_mis_list.append(i)

In [220]:
def processed(df):
    df = df.loc[df[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
    df['time_position_data_received'] = pd.to_datetime(df['time_position_data_received'])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.sort_values(by="timestamp", ignore_index=True,inplace=True)
    return df

In [221]:
# 억까 방지
busan_berth = '128.7362432822305,34.91571835540296,5.941836244531453 128.8816505843355,34.93956159255044,9.088787041386029 128.8718190496849,35.09310779849003,8.308118991212194 128.7247817257968,35.08647052441917,-9.228016031556635 128.7362432822305,34.91571835540296,5.941836244531453'
coord_list = busan_berth.split(" ")
coord_list
coord_list2 = []
for i in coord_list:
    temp = i.split(",")[:2]
    temp = list(map(float, temp))
    coord_list2.append(temp)
poly_busan = Polygon(coord_list2)
# if Point(LONGITUDE, LATITUDE).within(poly):

In [222]:
# ATA 확인
def check_ATA(df_test):
    try:
        stop_index = df_test[df_test.apply(lambda row: Point(
            row['longitude'], row['latitude']).within(poly_busan),
                                           axis=1)].index[0] - 1
        return stop_index
    except:
        return None

In [234]:
data_dict= {
    "vessel_name" : [],
    "voyage_name" : [],
    "ATB(port_mis)" : [],
    "ATB(AIS)" : [],
    "waiting_time" : []
}
for i in port_mis_list:
    port_mis_df = pd.read_csv(f"voy_path/{i}")
    vessel_name = i.split('_portmis')[0]
    for idx in port_mis_df.index:
        voyage_name = port_mis_df.loc[idx, "voyage_name"]
        port_in_time = port_mis_df.loc[idx, "port_in_time"]
        df = pd.read_csv(f'voy_path/{vessel_name}_{voyage_name}.csv') 
        if len(df) == 0:
            continue
        df = df.loc[df[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
        df['time_position_data_received'] = pd.to_datetime(df['time_position_data_received'])
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.sort_values(by="timestamp", ignore_index=True,inplace=True)
        max_timestamp = pd.to_datetime(port_in_time) + datetime.timedelta(hours=2)
        min_timestamp = pd.to_datetime(port_in_time) - datetime.timedelta(days=4)
        df_2 = df.query('timestamp >= @min_timestamp and timestamp < @max_timestamp')
        df_2.reset_index(inplace=True, drop=True)
        df_2['waiting_time'] = abs(df_2["timestamp"] - pd.to_datetime(port_in_time))
        stop_index = check_ATA(df_2)
        if stop_index == None:
            continue
        df_2 = df_2.loc[stop_index-30:]
        try:
            iiii = df_2[df_2["speed"] < 5].index[0]
        except:
            continue
        data_dict["vessel_name"].append(vessel_name)
        data_dict["voyage_name"].append(voyage_name)
        data_dict["ATB(port_mis)"].append(port_in_time)
        data_dict["ATB(AIS)"].append(df_2.iloc[df_2['waiting_time'].argmin()]["timestamp"])
        data_dict["waiting_time"].append(df_2.loc[iiii,"waiting_time"])
        
display(pd.DataFrame(data_dict))
#         df_2.to_csv('230117_test.csv',header=True, index=False, encoding="utf-8 sig")

,vessel_name,voyage_name,ATB(port_mis),ATB(AIS),waiting_time
0,AKADIMOS,9HA5144_2022_3,2022-12-18 21:10:00,2022-12-18 21:07:06.527387,0 days 00:32:54.574964
1,AKA_BHUM,9V7282_2022_6,2022-09-13 11:00:00,2022-09-13 10:44:15.737640,0 days 23:25:23.149609
2,AKA_BHUM,9V7282_2022_8,2022-12-27 06:55:00,2022-12-27 06:52:01.719709,0 days 01:13:08.703897
3,AMALTHEA,CQDE_2022_1,2022-11-08 14:40:00,2022-11-08 14:38:49.479124,0 days 00:11:15.933230
4,CAUQUENES,D5HF2_2022_3,2022-09-25 17:00:00,2022-09-25 16:56:01.462287,0 days 01:34:06.738383
...,...,...,...,...,...
218,ZHU_CHENG_XIN_ZHOU,BTSJ_2022_42,2022-12-17 04:05:00,2022-12-17 04:07:37.657174,0 days 01:12:33.642229
219,ZHU_CHENG_XIN_ZHOU,BTSJ_2022_43,2022-12-27 00:58:00,2022-12-27 00:56:08.702042,0 days 06:07:46.789281
220,ZHU_CHENG_XIN_ZHOU,BTSJ_2023_2,2023-01-10 23:47:00,2023-01-10 23:43:03.830417,0 days 00:23:59.238621
221,ZIM_CHARLESTON,VRUC3_2022_6,2022-10-26 22:10:00,2022-10-26 22:17:28.628546,0 days 00:27:50.458291


In [240]:
pd.DataFrame(data_dict).sort_values(by="waiting_time")

,vessel_name,voyage_name,ATB(port_mis),ATB(AIS),waiting_time
142,MSC_SARA_ELENA,CQEV8_2022_3,2022-08-27 16:10:00,2022-08-27 16:10:53.559000,0 days 00:00:53.559000
167,RIO_DE_JANEIRO_EXPRESS,DGXS2_2022_2,2022-09-16 02:03:00,2022-09-16 02:04:02.897656,0 days 00:01:02.897656
159,ONE_ORINOCO,VRUN3_2022_1,2022-09-17 08:12:00,2022-09-17 08:10:50.709638,0 days 00:01:09.290362
36,HMM_BLESSING,D7EW_2022_4,2022-10-13 03:12:00,2022-10-13 03:09:58.460099,0 days 00:02:01.539901
99,MSC_JUDITH,3EGR5_2022_1,2022-10-07 21:45:00,2022-10-07 21:47:01.936716,0 days 00:02:01.936716
...,...,...,...,...,...
87,MSC_GIANNINA_II,3FIP4_2022_9,2022-09-22 08:59:00,2022-09-22 08:54:55.609955,2 days 03:08:20.802835
52,MSC_ANDREA_F,D5WP9_2022_4,2022-09-15 04:07:00,2022-09-15 04:06:58.664608,2 days 11:43:23.190083
158,ONE_AMAZON,VRUN4_2022_1,2022-09-13 11:35:00,2022-09-13 12:14:16.871193,2 days 13:25:14.801272
81,MSC_EXPRESS_III,A8LG7_2022_1,2022-10-05 09:30:00,2022-10-05 09:41:13.132647,2 days 20:25:38.890562


In [250]:
df = pd.DataFrame(data_dict)

In [251]:
df.sort_values(by="waiting_time")["waiting_time"].mean()

Timedelta('0 days 05:36:02.576572928')

In [252]:
df.sort_values(by="waiting_time")["waiting_time"].max()

Timedelta('3 days 01:56:10.841108')